## Set up

In [1]:
import os
base_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir) 

In [ ]:
# import requests # request img from web
# import shutil # save img locally
# from pathlib import Path
from PIL import Image
# import numpy as np
# import math
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms

import utilities

In [ ]:
import imp
import extract_activations 
imp.reload(utilities)
imp.reload(extract_activations) 

In [ ]:
import sys
sys.path.append(f'{base_dir}/src/captum')
from captum import optim as optimviz

## Analyse activations and similarity

In [ ]:
activations_cat_dict = {}
activations_car_dict = {}
activations_cat_avg_dict = {}
activations_car_avg_dict = {}

activations_cat_avg_dot_dict = {}
activations_car_avg_dot_dict = {}
activations_cat_car_avg_dot_dict = {}

activations_cat_avg_sim_dict = {}
activations_car_avg_sim_dict = {}
activations_cat_car_avg_sim_dict = {}

### Extract activations, calculate averages, dot products

In [ ]:
# Change layer name here and run
layer = 'mixed4e.conv_1x1'

In [ ]:
activations_cat_avg = activations_cat.mean(3).mean(2) 
activations_car_avg = activations_car.mean(3).mean(2) 

In [ ]:
# np.dot(activations_car_avg[0], activations_car_avg[0])/(np.linalg.norm(activations_car_avg[0])*np.linalg.norm(activations_car_avg[0])) 

In [ ]:
activations_cat_avg_dot = np.empty([len(activations_cat_avg),len(activations_cat_avg)])
activations_cat_avg_sim = np.empty([len(activations_cat_avg),len(activations_cat_avg)])
for i in range(len(activations_cat_avg)):
    for j in range(len(activations_cat_avg)):
        activations_cat_avg_dot[i,j] = np.dot(activations_cat_avg[i], activations_cat_avg[j])
        activations_cat_avg_sim[i,j] = activations_cat_avg_dot[i,j]/(np.linalg.norm(activations_cat_avg[i])*np.linalg.norm(activations_cat_avg[j])) 
        
activations_car_avg_dot = np.empty([len(activations_car_avg),len(activations_car_avg)])
activations_car_avg_sim = np.empty([len(activations_car_avg),len(activations_car_avg)])
for i in range(len(activations_car_avg)):
    for j in range(len(activations_car_avg)):
        activations_car_avg_dot[i,j] = np.dot(activations_car_avg[i], activations_car_avg[j])
        activations_car_avg_sim[i,j] = activations_car_avg_dot[i,j]/(np.linalg.norm(activations_car_avg[i])*np.linalg.norm(activations_car_avg[j])) 
        
activations_cat_car_avg_dot = np.empty([len(activations_cat_avg),len(activations_car_avg)])
activations_cat_car_avg_sim = np.empty([len(activations_cat_avg),len(activations_car_avg)])
for i in range(len(activations_cat_avg)):
    for j in range(len(activations_car_avg)):
        activations_cat_car_avg_dot[i,j] = np.dot(activations_cat_avg[i], activations_car_avg[j])
        activations_cat_car_avg_sim[i,j] = activations_cat_car_avg_dot[i,j]/(np.linalg.norm(activations_cat_avg[i])*np.linalg.norm(activations_car_avg[j])) 


### Save

In [ ]:
activations_cat_dict[layer] = activations_cat
activations_car_dict[layer] = activations_car
activations_cat_avg_dict[layer] = activations_cat_avg
activations_car_avg_dict[layer] = activations_car_avg

activations_cat_avg_dot_dict[layer] = activations_cat_avg_dot
activations_car_avg_dot_dict[layer] = activations_car_avg_dot
activations_cat_car_avg_dot_dict[layer] = activations_cat_car_avg_dot

activations_cat_avg_sim_dict[layer] = activations_cat_avg_sim
activations_car_avg_sim_dict[layer] = activations_car_avg_sim
activations_cat_car_avg_sim_dict[layer] = activations_cat_car_avg_sim

### Analyse activations from some layer

In [ ]:
layer = 'mixed4e.conv_1x1'

In [ ]:
print(activations_car_dict[layer][0].mean(2).mean(1)[55])
print(activations_car_dict[layer][1].mean(2).mean(1)[55])
print(activations_car_dict[layer][-1].mean(2).mean(1)[55])
activation[layer][0].mean(2).mean(1)

Outputs 256 14x14 feature maps. 
How to visualise/analyse this shape? 
Create loop and average?? Look at unit 55. 

In [ ]:
# car hood: imagenet 81, flickr 61, 76, 81, 98

In [ ]:
# activations_cat_dict['mixed4e.conv_1x1'].shape # torch.Size([17, 256, 14, 14])
# activations_car_dict['mixed4e.conv_1x1'].shape # torch.Size([5, 256, 14, 14])

In [ ]:
# len(activations_cat_dict['mixed4e.conv_1x1']) # 17
# len(activations_car_dict['mixed4e.conv_1x1']) # 5
# activations_cat_avg_dict['mixed4e.conv_1x1'].shape # torch.Size([17, 256])
# activations_car_avg_dict['mixed4e.conv_1x1'].shape # torch.Size([5, 256])

In [ ]:
activations_cat_avg_dict[layer][0]

In [ ]:
print(activations_cat_dict[layer][0].mean(2).mean(1)[55])
print(activations_cat_dict[layer][1].mean(2).mean(1)[55])
print(activations_cat_dict[layer][-1].mean(2).mean(1)[55])

In [ ]:
activations_car_avg_dict[layer][2]

In [ ]:
plt.plot(activations_cat_avg_dict[layer][0]) # 42 peak

In [ ]:
plt.plot(activations_car_avg_dict[layer][0]) # 42 peak

### Cosine similarities

In [ ]:
cat = [f"cat{i}" for i in range(len(activations_cat_avg_dot_dict[layer]))]
car = [f"car{i}" for i in range(len(activations_car_avg_dot_dict[layer]))]

min_dot = min(np.amin(activations_cat_avg_dot_dict[layer]), np.amin(activations_car_avg_dot_dict[layer]), np.amin(activations_cat_car_avg_dot_dict[layer])) 
max_dot = max(np.amax(activations_cat_avg_dot_dict[layer]), np.amax(activations_car_avg_dot_dict[layer]), np.amax(activations_cat_car_avg_dot_dict[layer])) 

fig, axs = plt.subplots(3,1,figsize=(6, 12))

im1 = axs[0].imshow(activations_cat_avg_dot_dict[layer], cmap='viridis', interpolation='nearest', vmin=min_dot, vmax=max_dot)
axs[0].set_xticks(np.arange(len(activations_cat_avg_dot_dict[layer])), labels=cat)
axs[0].set_yticks(np.arange(len(activations_cat_avg_dot_dict[layer])), labels=cat)
axs[0].set_title("activations_cat_avg_dot")

im2 = axs[1].imshow(activations_car_avg_dot_dict[layer], cmap='viridis', interpolation='nearest', vmin=min_dot, vmax=max_dot)
axs[1].set_xticks(np.arange(len(activations_car_avg_dot_dict[layer])), labels=car)
axs[1].set_yticks(np.arange(len(activations_car_avg_dot_dict[layer])), labels=car)
axs[1].set_title("activations_car_avg_dot")
for i in range(len(activations_car_avg_dot_dict[layer])):
    for j in range(len(activations_car_avg_dot_dict[layer])):
        text = axs[1].text(j, i, activations_car_avg_dot_dict[layer][i, j]//1000,
                       ha="center", va="center", color="w")
        
im2 = axs[2].imshow(activations_cat_car_avg_dot_dict[layer], cmap='viridis', interpolation='nearest', vmin=min_dot, vmax=max_dot)
axs[2].set_xticks(np.arange(len(activations_car_avg_dot_dict[layer])), labels=car)
axs[2].set_yticks(np.arange(len(activations_cat_avg_dot_dict[layer])), labels=cat)
axs[2].set_title("activations_cat_car_avg_dot")

plt.subplots_adjust(right=0.8)
cbar_ax = plt.axes([0.85, 0.1, 0.075, 0.8])
plt.colorbar(mappable=(im1), cax=cbar_ax)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

plt.show()

In [ ]:
cat = [f"cat{i}" for i in range(len(activations_cat_avg_sim_dict[layer]))]
car = [f"car{i}" for i in range(len(activations_car_avg_sim_dict[layer]))]

min_sim = 0
max_sim = 1

fig, axs = plt.subplots(3,1,figsize=(6, 12))

im1 = axs[0].imshow(activations_cat_avg_sim_dict[layer], cmap='viridis', interpolation='nearest', vmin=min_sim, vmax=max_sim) 
axs[0].set_xticks(np.arange(len(activations_cat_avg_sim_dict[layer])), labels=cat)
axs[0].set_yticks(np.arange(len(activations_cat_avg_sim_dict[layer])), labels=cat)
axs[0].set_title("activations_cat_avg_sim")

im2 = axs[1].imshow(activations_car_avg_sim_dict[layer], cmap='viridis', interpolation='nearest', vmin=min_sim, vmax=max_sim) 
axs[1].set_xticks(np.arange(len(activations_car_avg_sim_dict[layer])), labels=car)
axs[1].set_yticks(np.arange(len(activations_car_avg_sim_dict[layer])), labels=car)
axs[1].set_title("activations_car_avg_sim")
for i in range(len(activations_car_avg_sim_dict[layer])):
    for j in range(len(activations_car_avg_sim_dict[layer])):
        text = axs[1].text(j, i, round(activations_car_avg_sim_dict[layer][i, j],2),
                       ha="center", va="center") # , color="w"
        
im2 = axs[2].imshow(activations_cat_car_avg_sim_dict[layer], cmap='viridis', interpolation='nearest', vmin=min_sim, vmax=max_sim) 
axs[2].set_xticks(np.arange(len(activations_car_avg_sim_dict[layer])), labels=car)
axs[2].set_yticks(np.arange(len(activations_cat_avg_sim_dict[layer])), labels=cat)
axs[2].set_title("activations_cat_car_avg_sim")

plt.subplots_adjust(right=0.8)
cbar_ax = plt.axes([0.85, 0.1, 0.075, 0.8])
plt.colorbar(mappable=(im1), cax=cbar_ax)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

plt.show()

## Other

Another model...

In [ ]:
activations_cat = torch.empty(0,256,14,14)
activations_car = torch.empty(0,256,14,14)
directory_cat = f'{base_dir}/data/test-images/cat'
directory_car = f'{base_dir}/data/test-images/car'

for file in os.listdir(directory_cat): # os.listdir(directory)
    activation = {}
    input_image = Image.open(directory_cat+'/'+file).convert('RGB') # since 'RGBA'
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)

    googlenet_torchvision.inception4e.branch1.conv.register_forward_hook(get_activation(layer, googlenet_torchvision))
    output = googlenet_torchvision(input_batch)
    activations_cat = torch.cat((activations_cat,activation[layer]))

for file in os.listdir(directory_car):
    activation = {}
    input_image = Image.open(directory_car+'/'+file).convert('RGB') # since 'RGBA'
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)

    googlenet_torchvision.inception4e.branch1.conv.register_forward_hook(get_activation(layer, googlenet_torchvision))
    output = googlenet_torchvision(input_batch)
    activations_car = torch.cat((activations_car,activation[layer]))